In [115]:
import numpy as np
import pandas as pd
import random as rd

In [116]:
#CREAMOS LOS REGISTROS SINTETICOS

#diccionario con 104 registros de tarifa con 20 registros cada una de total de kWh consumido en el periodo de facturacion

np.random.seed = 1309

consumos = {}

for tarifa in range(104):
    consumos[tarifa] = []
    for registro in range(21):
        valores = np.random.randint(100, 300)
        consumos[tarifa].append(valores)

#diccionario con registros de tarifa con 104 registros cada una de pesos de consumo para la pvpc y tarifas por tramos


pesos_3t = {}
for tarifa in range(104):
    pesos_3t[tarifa] = []
    while len(pesos_3t[tarifa]) < 21:
        a,b = np.random.randint(0,101,2)
        c = 100 - a - b
        while c < 0:
            a,b = np.random.randint(0,101,2)
            c = 100 - a - b
        pesos_3t[tarifa].append([a,b,c])

#diccionario con registros de tarifa con 105 registros cada una de pesos de consumo para tarifas de discriminacion horaria


pesos_2t = {}
for tarifa in range(104):
    pesos_2t[tarifa] = []
    while len(pesos_2t[tarifa]) < 21:
        d = np.random.randint(0,101)
        e = 100 - d
        pesos_2t[tarifa].append([d,e])

In [117]:
#APLANAMOS LOS DICCIONARIOS

registros = []
for tarifa, valores in consumos.items():
    for valor in valores:
        registros.append({'tarifa': tarifa, 'consumo_kWh': valor})

tramos_3t = []
for tarifa, valores in pesos_3t.items():
    for valor in valores:
        tramos_3t.append({'tarifa': tarifa, 'pesos': valor})

tramos_2t = []
for tarifa, valores in pesos_2t.items():
    for valor in valores:
        tramos_2t.append({'tarifa': tarifa, 'pesos': valor})

In [118]:
totales = pd.DataFrame(registros)

In [119]:
totales['%consumo_valle'] = None
totales['%consumo_llano'] = None
totales['%consumo_punta'] = None
totales['%consumo_8h'] = None
totales['%consumo_16h'] = None

In [120]:
for index, i in enumerate(tramos_3t):
        totales.loc[index, '%consumo_valle'] = i['pesos'][0]
        totales.loc[index, '%consumo_llano'] = i['pesos'][1]
        totales.loc[index, '%consumo_punta'] = i['pesos'][2]

for index, i in enumerate(tramos_2t):
        totales.loc[index, '%consumo_8h'] = i['pesos'][0]
        totales.loc[index, '%consumo_16h'] = i['pesos'][1]

In [121]:
totales

,tarifa,consumo_kWh,%consumo_valle,%consumo_llano,%consumo_punta,%consumo_8h,%consumo_16h
0,0,237,7,67,26,93,7
1,0,210,47,33,20,56,44
2,0,264,46,33,21,54,46
3,0,274,58,7,35,39,61
4,0,197,11,3,86,29,71
...,...,...,...,...,...,...,...
2179,103,197,88,5,7,73,27
2180,103,257,36,58,6,25,75
2181,103,167,83,15,2,3,97
2182,103,126,47,46,7,48,52


In [122]:
totales = totales.rename(columns={'tarifa': 'id_tarifa'})

In [126]:
# Traigo la base de datos
df = pd.read_csv('../Datos/database_desafio.csv',sep =';')

In [127]:
df.head()

,id,compania,tarifa,tp_punta_euros/kwh/dia,tp_valle_euros/kwh/dia,peaje_tran_dist_pot_valle,peaje_tran_dist_pot_punta,peaje_tran_dist_ener_valle,peaje_tran_dist_ener_llano,peaje_tran_dist_ener_punta,termino_consumo_euros/kwh/hora,tc_valle_euros/kwh/hora,tc_llano_euros/kwh/hora,tc_punta_euros/kwh/hora,tc_8h_euros/kwh/hora,tc_16h_euros/kwh/hora,mercado,tipo,otros,alquiler_equipo_euros/dia,impuesto_actual,bono_social_euro/dia
0,0.0,Repsol,Mis 10 Horas,0.068192,0.068192,0.000527,0.008192,0.002195,0.008779,0.043893,NaN,NaN,NaN,NaN,0.086415,0.17283,Libre,DH,NaN,0.026557,5.11,NaN
1,1.0,Iberdrola,Plan Elige 8 Horas,0.108192,0.042411,0.000527,0.008192,0.002195,0.008779,0.043893,NaN,NaN,NaN,NaN,0.096700,0.24100,Libre,DH,NaN,0.026557,5.11,NaN
2,2.0,Iberdrola,Plan Noche,0.108192,0.042411,0.000527,0.008192,0.002195,0.008779,0.043893,NaN,NaN,NaN,NaN,0.109000,0.23900,Libre,DH,NaN,0.026557,5.11,NaN
3,3.0,Iberdrola,Plan Familias,0.108192,0.042411,0.000527,0.008192,0.002195,0.008779,0.043893,NaN,NaN,NaN,NaN,0.103000,0.18900,Libre,DH,NaN,0.026557,5.11,NaN
4,4.0,Iberdrola,Plan Verano,0.108192,0.042411,0.000527,0.008192,0.002195,0.008779,0.043893,NaN,NaN,NaN,NaN,0.099000,0.21900,Libre,DH,NaN,0.026557,5.11,NaN


In [128]:
# Uno los datos sinteticos con las tarifas
clientes_tarifas = pd.merge(totales, df, left_on='id_tarifa', right_on='id', how='left')

In [129]:
# Para que se vean todas las columnas
pd.set_option('display.max_columns', None)

In [130]:
clientes_tarifas

,id_tarifa,consumo_kWh,%consumo_valle,%consumo_llano,%consumo_punta,%consumo_8h,%consumo_16h,id,compania,tarifa,tp_punta_euros/kwh/dia,tp_valle_euros/kwh/dia,peaje_tran_dist_pot_valle,peaje_tran_dist_pot_punta,peaje_tran_dist_ener_valle,peaje_tran_dist_ener_llano,peaje_tran_dist_ener_punta,termino_consumo_euros/kwh/hora,tc_valle_euros/kwh/hora,tc_llano_euros/kwh/hora,tc_punta_euros/kwh/hora,tc_8h_euros/kwh/hora,tc_16h_euros/kwh/hora,mercado,tipo,otros,alquiler_equipo_euros/dia,impuesto_actual,bono_social_euro/dia
0,0,237,7,67,26,93,7,0.0,Repsol,Mis 10 Horas,0.068192,0.068192,0.000527,0.008192,0.002195,0.008779,0.043893,NaN,NaN,NaN,NaN,0.086415,0.17283,Libre,DH,NaN,0.026557,5.11,NaN
1,0,210,47,33,20,56,44,0.0,Repsol,Mis 10 Horas,0.068192,0.068192,0.000527,0.008192,0.002195,0.008779,0.043893,NaN,NaN,NaN,NaN,0.086415,0.17283,Libre,DH,NaN,0.026557,5.11,NaN
2,0,264,46,33,21,54,46,0.0,Repsol,Mis 10 Horas,0.068192,0.068192,0.000527,0.008192,0.002195,0.008779,0.043893,NaN,NaN,NaN,NaN,0.086415,0.17283,Libre,DH,NaN,0.026557,5.11,NaN
3,0,274,58,7,35,39,61,0.0,Repsol,Mis 10 Horas,0.068192,0.068192,0.000527,0.008192,0.002195,0.008779,0.043893,NaN,NaN,NaN,NaN,0.086415,0.17283,Libre,DH,NaN,0.026557,5.11,NaN
4,0,197,11,3,86,29,71,0.0,Repsol,Mis 10 Horas,0.068192,0.068192,0.000527,0.008192,0.002195,0.008779,0.043893,NaN,NaN,NaN,NaN,0.086415,0.17283,Libre,DH,NaN,0.026557,5.11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179,103,197,88,5,7,73,27,103.0,Aduriz Energia,coste + fija,0.080039,0.003141,0.000527,0.008192,0.002195,0.008779,0.043893,NaN,0.093282,0.129895,0.179284,NaN,NaN,Libre,TRAMOS,3.72,0.026557,5.11,NaN
2180,103,257,36,58,6,25,75,103.0,Aduriz Energia,coste + fija,0.080039,0.003141,0.000527,0.008192,0.002195,0.008779,0.043893,NaN,0.093282,0.129895,0.179284,NaN,NaN,Libre,TRAMOS,3.72,0.026557,5.11,NaN
2181,103,167,83,15,2,3,97,103.0,Aduriz Energia,coste + fija,0.080039,0.003141,0.000527,0.008192,0.002195,0.008779,0.043893,NaN,0.093282,0.129895,0.179284,NaN,NaN,Libre,TRAMOS,3.72,0.026557,5.11,NaN
2182,103,126,47,46,7,48,52,103.0,Aduriz Energia,coste + fija,0.080039,0.003141,0.000527,0.008192,0.002195,0.008779,0.043893,NaN,0.093282,0.129895,0.179284,NaN,NaN,Libre,TRAMOS,3.72,0.026557,5.11,NaN


In [131]:
# comprobamos por tarifas si el merge es como debe

clientes_tarifas[clientes_tarifas['id_tarifa'] == 12]

,id_tarifa,consumo_kWh,%consumo_valle,%consumo_llano,%consumo_punta,%consumo_8h,%consumo_16h,id,compania,tarifa,tp_punta_euros/kwh/dia,tp_valle_euros/kwh/dia,peaje_tran_dist_pot_valle,peaje_tran_dist_pot_punta,peaje_tran_dist_ener_valle,peaje_tran_dist_ener_llano,peaje_tran_dist_ener_punta,termino_consumo_euros/kwh/hora,tc_valle_euros/kwh/hora,tc_llano_euros/kwh/hora,tc_punta_euros/kwh/hora,tc_8h_euros/kwh/hora,tc_16h_euros/kwh/hora,mercado,tipo,otros,alquiler_equipo_euros/dia,impuesto_actual,bono_social_euro/dia
252,12,216,22,66,12,67,33,12.0,Endesa,ConectaEndesa,0.099703,0.032791,0.000527,0.008192,0.002195,0.008779,0.043893,0.0957,NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,0.026557,5.11,NaN
253,12,177,68,26,6,44,56,12.0,Endesa,ConectaEndesa,0.099703,0.032791,0.000527,0.008192,0.002195,0.008779,0.043893,0.0957,NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,0.026557,5.11,NaN
254,12,235,50,27,23,41,59,12.0,Endesa,ConectaEndesa,0.099703,0.032791,0.000527,0.008192,0.002195,0.008779,0.043893,0.0957,NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,0.026557,5.11,NaN
255,12,228,40,53,7,79,21,12.0,Endesa,ConectaEndesa,0.099703,0.032791,0.000527,0.008192,0.002195,0.008779,0.043893,0.0957,NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,0.026557,5.11,NaN
256,12,132,31,38,31,17,83,12.0,Endesa,ConectaEndesa,0.099703,0.032791,0.000527,0.008192,0.002195,0.008779,0.043893,0.0957,NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,0.026557,5.11,NaN
257,12,140,11,11,78,41,59,12.0,Endesa,ConectaEndesa,0.099703,0.032791,0.000527,0.008192,0.002195,0.008779,0.043893,0.0957,NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,0.026557,5.11,NaN
258,12,175,73,3,24,14,86,12.0,Endesa,ConectaEndesa,0.099703,0.032791,0.000527,0.008192,0.002195,0.008779,0.043893,0.0957,NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,0.026557,5.11,NaN
259,12,109,78,22,0,71,29,12.0,Endesa,ConectaEndesa,0.099703,0.032791,0.000527,0.008192,0.002195,0.008779,0.043893,0.0957,NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,0.026557,5.11,NaN
260,12,242,4,46,50,83,17,12.0,Endesa,ConectaEndesa,0.099703,0.032791,0.000527,0.008192,0.002195,0.008779,0.043893,0.0957,NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,0.026557,5.11,NaN
261,12,107,25,13,62,36,64,12.0,Endesa,ConectaEndesa,0.099703,0.032791,0.000527,0.008192,0.002195,0.008779,0.043893,0.0957,NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,0.026557,5.11,NaN


In [132]:
clientes_tarifas.drop(columns = 'id',inplace=True)

In [134]:
clientes_tarifas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2184 entries, 0 to 2183
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id_tarifa                       2184 non-null   int64  
 1   consumo_kWh                     2184 non-null   int64  
 2   %consumo_valle                  2184 non-null   object 
 3   %consumo_llano                  2184 non-null   object 
 4   %consumo_punta                  2184 non-null   object 
 5   %consumo_8h                     2184 non-null   object 
 6   %consumo_16h                    2184 non-null   object 
 7   compania                        2184 non-null   object 
 8   tarifa                          2184 non-null   object 
 9   tp_punta_euros/kwh/dia          2184 non-null   float64
 10  tp_valle_euros/kwh/dia          2184 non-null   float64
 11  peaje_tran_dist_pot_valle       2184 non-null   float64
 12  peaje_tran_dist_pot_punta       21

In [135]:
int = clientes_tarifas.columns[2:7]

In [137]:
for x in int:
    if clientes_tarifas[x].dtype == 'object':
        clientes_tarifas[x] = clientes_tarifas[x].astype('int32') 


In [138]:
clientes_tarifas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2184 entries, 0 to 2183
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id_tarifa                       2184 non-null   int64  
 1   consumo_kWh                     2184 non-null   int64  
 2   %consumo_valle                  2184 non-null   int32  
 3   %consumo_llano                  2184 non-null   int32  
 4   %consumo_punta                  2184 non-null   int32  
 5   %consumo_8h                     2184 non-null   int32  
 6   %consumo_16h                    2184 non-null   int32  
 7   compania                        2184 non-null   object 
 8   tarifa                          2184 non-null   object 
 9   tp_punta_euros/kwh/dia          2184 non-null   float64
 10  tp_valle_euros/kwh/dia          2184 non-null   float64
 11  peaje_tran_dist_pot_valle       2184 non-null   float64
 12  peaje_tran_dist_pot_punta       21

In [ ]:
clientes_tarifas

In [ ]:
# CALCULOS DE TARIFA

#TERMINOS FIJOS IGUALES PARA TODOS

def pvpc():
    


    peaje_pot_punta = 
    peaje_pot_valle =
    peaje_ene_punta =
    peaje_ene_punta = 

#PVPC

    #termino fijo
    peaje_pot_punta = 
    peaje_pot_valle =
    peaje_ene_punta =
    peaje_ene_punta = 
#FIJA


#TRAMOS


#DISCRIMINACION HORARIA


